In [5]:
from gazebo_msgs.srv import SpawnModel,SetModelState
import rospy
from geometry_msgs.msg import Pose
from gazebo_msgs.msg import ModelState 
from random import randrange
import PyKDL
import numpy as np
import time

def Summon_box(box_X,box_Y,box_Z,box_R,box_P,box_Yaw,box_name):

    rot_quat = PyKDL.Rotation.RPY(np.radians(box_R), np.radians(box_P), np.radians(box_Yaw)).GetQuaternion() #0.328462 0 0.000036 0 -1.570604 3.141059
    box_pose = Pose()
    box_pose.position.x = box_X
    box_pose.position.y = box_Y
    box_pose.position.z = box_Z
    box_pose.orientation.x = rot_quat[0]
    box_pose.orientation.y = rot_quat[1]
    box_pose.orientation.z = rot_quat[2]
    box_pose.orientation.w = rot_quat[3]

    spawn_model_client = rospy.ServiceProxy('gazebo/spawn_sdf_model', SpawnModel)
    stat=spawn_model_client( model_name = box_name, model_xml=open('/home/shadowd/ws_panda/src/Box_test/model.sdf', 'r').read(),
        robot_namespace='', initial_pose=box_pose, reference_frame='world')

    return stat


def update_box_pose(hold_time, box_name, Task_pose):
    
    box_state_msg = ModelState()
    t_end = time.time() + hold_time

    while True:
        box_state_msg.model_name = box_name
        box_state_msg.pose.position.x = Task_pose[0]
        box_state_msg.pose.position.y = Task_pose[1] 
        box_state_msg.pose.position.z = Task_pose[2]

        box_rot_quat = PyKDL.Rotation.RPY(np.radians(Task_pose[3]), np.radians(Task_pose[4]), np.radians(Task_pose[5])).GetQuaternion()
        box_state_msg.pose.orientation.x = box_rot_quat[0]
        box_state_msg.pose.orientation.y = box_rot_quat[1]
        box_state_msg.pose.orientation.z = box_rot_quat[2]
        box_state_msg.pose.orientation.w = box_rot_quat[3]
        resp = set_state( box_state_msg )
        if(time.time() > t_end):
            break
    return resp 

rospy.init_node('insert_object',log_level=rospy.INFO)
rospy.wait_for_service('/gazebo/spawn_sdf_model')
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)


In [9]:
box_name = 'Test_box_'+ str(101)
box_X = 0.3
box_Y = 0.0
box_Z = 0.1

box_R = 45
box_P = 0
box_Yaw = 0

Summon_box(box_X,box_Y,box_Z,box_R,box_P,box_Yaw,box_name)

success: True
status_message: "SpawnModel: Successfully spawned entity"

In [10]:
Box_pose = [0.3, 0, 0.15, 0, 0, 0]

update_box_pose(1, box_name, Box_pose)

success: True
status_message: "SetModelState: set model state done"